# Example - using Mixture of Experts layer to define Keras model

This example illustrates a practical application how multiple experts can be combined together to build an effective mixture model. 

In this example, a gating model (defined a single-layer neural network with noisy version of softmax activation), would channel each example to exactly 2 experts, and combine their results with corresponding weight.

In [1]:
import numpy as np
import keras
import tensorflow as tf
import time
import tensor2tensor
from tensor2tensor.utils.expert_utils import *
from customutils.CustomLayers import MixtureOfExpertsLayer
import keras
import keras.backend as K

c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Generate synthetic data for practice
X = np.random.normal(scale = 1, size = 10**6).reshape(-1,20) #50K observations and 20 variables
y = X[:,0] + X[:,1] + X[:,0]* X[:,1]
X.shape, y.shape
print(np.var(y))


3.0329414499386727


In [3]:
# define model architecture that would be considered as single expert
class TimeLimit(keras.callbacks.Callback):
    def __init__(self, time_limit_seconds, verbose = 1):
        self.time_limit_seconds = time_limit_seconds
        self.verbose = verbose
        
    def on_train_begin(self,logs={}):
        self.time_start = time.time()
        
    def on_epoch_end(self,epoch,logs):
        if time.time() >= self.time_start + self.time_limit_seconds:
            self.model.stop_training = True
            if self.verbose > 0: print('Training terminated after epoch ', epoch+1)

        
callbacks_list = []

def build_model(input_dim = 20, hidden_layer_sizes = [4,1], lr_init = 0.1, lr_decay = 0.001, activation = 'tanh', output_dim = 1, random_state = 7, early_stopping = False, max_fit_time = 60, compile = False):
    tf.set_random_seed(random_state)
    np.random.seed(random_state)
    inputs  = keras.layers.Input((input_dim,), dtype='float32')
    for idx, hidden_layer_size in enumerate(hidden_layer_sizes):
        if idx == 0: hidden = keras.layers.Dense(hidden_layer_size, activation = activation)(inputs)
        else:        hidden = keras.layers.Dense(hidden_layer_size, activation = activation)(hidden)
    outputs = keras.layers.Dense(output_dim)(hidden) # no activation for the last layer
    model   = keras.Model(inputs, outputs)
    #model.summary()
    if compile ==True:
        model.compile(loss = 'mean_squared_error', optimizer = keras.optimizers.Adam(lr=lr_init, decay = lr_decay))

    # add callbacks to be utilized by the fit function - could check if 'callbacks_list' not in globals() 
    global callbacks_list 
    if 'callbacks_list' in globals(): callbacks_list.clear()
    
    if early_stopping == True:
        callbacks_list.append(keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=2, verbose=1, mode='auto'))
    if max_fit_time: 
        callbacks_list.append(TimeLimit(max_fit_time))
    
    return model  

In [4]:
def build_MoE_as_model(n_experts, input_dim, output_dim, model_function, use_top_K = None, loss = 'mse', optimizer = 'rmsprop', sparse_inputs = False, random_state = 7):
    # initialize models    
    models = []
    for i in range(n_experts):
        with tf.name_scope('Expert_' + str(1+i)):
            model = model_function(random_state = 13 + i * random_state) # creates a separate expert
            models.append(model)
    
    #inputs = keras.layers.Input((input_dim,), dtype='float32')
    inputs = keras.layers.Input((input_dim,), dtype='float', sparse = sparse_inputs)
    outputs= MixtureOfExpertsLayer(models, use_top_K = use_top_K, input_dim = input_dim)(inputs)
    model  = keras.Model(inputs, outputs)
    model.compile(loss = loss, optimizer = optimizer)
    
    #could also consider returning KerasRegressor wrapper
    return model

In [5]:
K.clear_session()
keras_model = build_MoE_as_model(
        10,                                                                   # number of experts to train
        20,                                                                   # input dimensions
        1,                                                                    # output dimensions 
        lambda **kwargs: build_model(hidden_layer_sizes = [20,10], **kwargs), # individual experts
        use_top_K = 2,                                                        # number of experts to use for each record 
        optimizer = keras.optimizers.Adam(lr = 0.01),                         # optimizer
        sparse_inputs = False                                                 # whether each model would use sparse inputs  
)
keras_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 20)                0         
_________________________________________________________________
mixture_of_experts_layer_1 ( ((None,), 1)              6410      
Total params: 6,410
Trainable params: 6,410
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Fit model to the synthetic data generated above
keras_model.fit(X,y, batch_size = 2**10, epochs = 100)

Epoch 1/100
50000/50000 [==============================] - 3s 68us/step - loss: 1.3213
Epoch 2/100
50000/50000 [==============================] - 1s 24us/step - loss: 0.6242
Epoch 3/100
50000/50000 [==============================] - 1s 23us/step - loss: 0.2034
Epoch 4/100
50000/50000 [==============================] - 1s 17us/step - loss: 0.0951
Epoch 5/100
50000/50000 [==============================] - 1s 17us/step - loss: 0.0619
Epoch 6/100
50000/50000 [==============================] - 1s 19us/step - loss: 0.0469
Epoch 7/100
50000/50000 [==============================] - 1s 15us/step - loss: 0.0372
Epoch 8/100
50000/50000 [==============================] - 1s 22us/step - loss: 0.0300
Epoch 9/100
50000/50000 [==============================] - 1s 16us/step - loss: 0.0254
Epoch 10/100
50000/50000 [==============================] - 1s 22us/step - loss: 0.0211: 0s - lo
Epoch 11/100
50000/50000 [==============================] - 1s 20us/step - loss: 0.0180
Epoch 12/100
50000/50000 [======

50000/50000 [==============================] - 1s 21us/step - loss: 0.0045
Epoch 95/100
50000/50000 [==============================] - 1s 21us/step - loss: 0.0046
Epoch 96/100
50000/50000 [==============================] - 1s 26us/step - loss: 0.0046
Epoch 97/100
50000/50000 [==============================] - 1s 25us/step - loss: 0.0044
Epoch 98/100
50000/50000 [==============================] - 1s 18us/step - loss: 0.0052
Epoch 99/100
50000/50000 [==============================] - 1s 16us/step - loss: 0.0061
Epoch 100/100
50000/50000 [==============================] - 1s 15us/step - loss: 0.0056


In [7]:
# Report model fit metrics
import sklearn
sklearn.metrics.r2_score(keras_model.predict(X, batch_size = 10**20), y)

0.9987138291250041

This model is attaining about 99.9% $R^2$ after less than 100 epochs (about 1 second per epoch on GPU)